In [2]:
import numpy as np
import pickle

In [21]:
data = None
column_names = ['long', 'sweet', 'yellow', 'seed', 'Brazil']

with open('fruit_data.pkl', 'rb') as fin:
    data = pickle.load(fin)

In [19]:
data

array([['others', '0', '1', '0', '1', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ...,
       ['bananas', '1', '1', '1', '0', '0'],
       ['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0']], dtype='<U21')

## Prior

In [12]:
data[:, :1]

array([['others'],
       ['bananas'],
       ['bananas'],
       ...,
       ['bananas'],
       ['oranges'],
       ['oranges']], dtype='<U21')

In [16]:
pre_priors = np.unique(data[:, :1], return_counts=True) # <- magic, this almost returns our prior probabilities
pre_priors[1]/len(data)

array([0.28571429, 0.21428571, 0.35714286, 0.14285714])

In [57]:
labels

array(['bananas', 'mangoes', 'oranges', 'others'], dtype='<U21')

In [18]:
labels = pre_priors[0]
priors = pre_priors[1]/len(data)

This was easy 👌

## Posterior

In [30]:
banana_indices = np.where(data[:, :1] == 'bananas')[0] # <- all row indices where there is data for bananas
banana_indices

array([   1,    2,    3,    5,    6,    7,   11,   16,   18,   20,   22,
         25,   26,   29,   44,   47,   50,   51,   52,   53,   54,   56,
         60,   61,   65,   66,   68,   69,   73,   75,   76,   84,   96,
         99,  101,  108,  112,  123,  124,  126,  128,  131,  133,  134,
        135,  142,  146,  148,  150,  152,  154,  155,  161,  165,  166,
        167,  173,  174,  180,  181,  185,  186,  187,  193,  201,  204,
        205,  207,  214,  215,  220,  225,  226,  230,  233,  234,  239,
        242,  255,  260,  261,  265,  266,  270,  272,  275,  288,  291,
        294,  296,  305,  306,  307,  311,  314,  318,  319,  320,  324,
        326,  327,  334,  342,  344,  348,  363,  364,  365,  366,  367,
        369,  370,  376,  378,  380,  381,  385,  390,  391,  396,  401,
        402,  403,  407,  408,  413,  415,  416,  417,  419,  420,  421,
        422,  424,  426,  434,  437,  438,  441,  444,  448,  455,  459,
        464,  468,  473,  474,  477,  480,  484,  4

In [29]:
bananas = data[banana_indices]
bananas

array([['bananas', '1', '0', '1', '0', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ['bananas', '0', '1', '1', '0', '1'],
       ...,
       ['bananas', '1', '1', '1', '0', '0'],
       ['bananas', '1', '1', '1', '0', '0'],
       ['bananas', '1', '1', '1', '0', '0']], dtype='<U21')

In [31]:
orange_indices = np.where(data[:, :1] == 'oranges')[0]
oranges = data[orange_indices]
oranges

array([['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0'],
       ...,
       ['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0']], dtype='<U21')

In [32]:
mango_indices = np.where(data[:, :1] == 'mangoes')[0]
mangoes = data[mango_indices]
mangoes

array([['mangoes', '0', '1', '1', '0', '1'],
       ['mangoes', '0', '0', '0', '0', '1'],
       ['mangoes', '1', '1', '1', '0', '1'],
       ...,
       ['mangoes', '0', '1', '1', '0', '1'],
       ['mangoes', '1', '1', '1', '0', '1'],
       ['mangoes', '0', '0', '0', '0', '1']], dtype='<U21')

In [33]:
other_indices = np.where(data[:, :1] == 'others')[0]
others = data[other_indices]
others

array([['others', '0', '1', '0', '1', '0'],
       ['others', '1', '1', '0', '1', '0'],
       ['others', '1', '1', '0', '1', '0'],
       ...,
       ['others', '0', '1', '0', '1', '0'],
       ['others', '0', '1', '1', '1', '0'],
       ['others', '0', '1', '1', '1', '0']], dtype='<U21')

In [37]:
# let's create our posterio matrix
# row names -> bananas, mangoes, oranges, others
# column indices -> long, NOT long, sweet, NOT sweet, yellow, NOT yellow, sweet, NOT sweet, Brazil, NOT Brazil
# a 4x10 matrix

posteriors = np.zeros([4, 10])
posteriors

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [40]:
data

array([['others', '0', '1', '0', '1', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ['bananas', '1', '0', '1', '0', '0'],
       ...,
       ['bananas', '1', '1', '1', '0', '0'],
       ['oranges', '0', '1', '0', '1', '0'],
       ['oranges', '0', '1', '0', '1', '0']], dtype='<U21')

In [56]:
# populate the posterior matrix

# 1. bananas, e.g. posteriors matrix row 0
int_feature = bananas[:, 1:2].astype('int') # <- which bananas are long - True or False
int_feature.sum() # <- the count of long bananas
len(bananas[:, 1:2]) - int_feature.sum() # <- the count of NOT long bananas

40

In [94]:
def populate_posterior_row(ind, fruit_data_slice):
    j = 0 
    for i in range(len(column_names)+1):

        # skips the first column, which holds the names of the fruits
        if i == 0:
            continue


        int_feature = fruit_data_slice[:, i:i+1].astype('int')
        feature_count = int_feature.sum() / len(fruit_data_slice)
        not_feature_count = (len(fruit_data_slice[:, i:i+1]) - int_feature.sum()) / len(fruit_data_slice)

        posteriors[ind, j] = feature_count
        j += 1
        posteriors[ind, j] = not_feature_count
        j += 1

In [97]:
populate_posterior_row(0, bananas)
populate_posterior_row(1, mangoes)
populate_posterior_row(2, oranges)
populate_posterior_row(3, others)

In [98]:
posteriors

array([[0.9       , 0.1       , 0.625     , 0.375     , 0.975     ,
        0.025     , 0.        , 1.        , 0.1625    , 0.8375    ],
       [0.36666667, 0.63333333, 0.66666667, 0.33333333, 0.4       ,
        0.6       , 0.        , 1.        , 1.        , 0.        ],
       [0.006     , 0.994     , 0.92      , 0.08      , 0.06      ,
        0.94      , 0.94      , 0.06      , 0.02      , 0.98      ],
       [0.225     , 0.775     , 1.        , 0.        , 0.5       ,
        0.5       , 0.95      , 0.05      , 0.        , 1.        ]])